## Example Usage the HecTime Class ##

### Setup ###

In [13]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from typing import cast

from IPython.display import display_html

from hec import HecTime, Interval, TimeSpan

### End-of-month 1Month intervals for a time window ###

In [14]:
intvl = Interval.get_dss("1Month")
start_time = HecTime("2024-01-31, 24:00")
end_time = start_time + "1Y"
count = start_time.compute_number_intervals(end_time, intvl)
times = [start_time + f"{i}M" for i in range(count)]
for t in times:
    print(t)

2024-01-31T24:00:00
2024-02-29T24:00:00
2024-03-31T24:00:00
2024-04-30T24:00:00
2024-05-31T24:00:00
2024-06-30T24:00:00
2024-07-31T24:00:00
2024-08-31T24:00:00
2024-09-30T24:00:00
2024-10-31T24:00:00
2024-11-30T24:00:00
2024-12-31T24:00:00


### LRTS Daily Times in UTC ###
Note the transition from 13:00 to 12:00 (23 hour day) in the Spring DST transition and then back again (25 hour day) in the Autum DST transistion.

In [ ]:
opt_intvl = Interval.get_any_cwms(lambda i: i.minutes == 1440 and i.is_local_regular)
assert opt_intvl is not None
intvl = opt_intvl
offset = 7 * Interval.MINUTES["1Hour"]
for month in ("Mar", "Nov"):
    start_time = HecTime(f"01{month}2024, 00:00").label_as_time_zone("US/Central")
    end_time = start_time + "P1M"
    first_time = HecTime(start_time)
    if first_time.adjust_to_interval_offset(intvl, offset) < start_time:
        first_time.increment(1, intvl)
    count = first_time.compute_number_intervals(end_time, intvl)
    this_time = first_time
    times = [this_time.copy()]
    for i in range(1, count):
        times.append(this_time.increment(1, intvl).copy())
    prev = None
    for t in times:
        utc = t.copy().convert_to_time_zone("UTC")
        if prev:
            hours_diff = cast(TimeSpan, (t - prev)).total_seconds() // 3600
            if hours_diff == 24:
                print(f"{t}\t{utc}\t{hours_diff}")
            else:
                print(f"{t}\t{utc}\t{hours_diff} <==")
        else:
            print(f"{t}\t{utc}")
        prev = t
    print("")

2024-03-01T07:00:00-06:00	2024-03-01T13:00:00+00:00
2024-03-02T07:00:00-06:00	2024-03-02T13:00:00+00:00	24
2024-03-03T07:00:00-06:00	2024-03-03T13:00:00+00:00	24
2024-03-04T07:00:00-06:00	2024-03-04T13:00:00+00:00	24
2024-03-05T07:00:00-06:00	2024-03-05T13:00:00+00:00	24
2024-03-06T07:00:00-06:00	2024-03-06T13:00:00+00:00	24
2024-03-07T07:00:00-06:00	2024-03-07T13:00:00+00:00	24
2024-03-08T07:00:00-06:00	2024-03-08T13:00:00+00:00	24
2024-03-09T07:00:00-06:00	2024-03-09T13:00:00+00:00	24
2024-03-10T07:00:00-05:00	2024-03-10T12:00:00+00:00	23 <==
2024-03-11T07:00:00-05:00	2024-03-11T12:00:00+00:00	24
2024-03-12T07:00:00-05:00	2024-03-12T12:00:00+00:00	24
2024-03-13T07:00:00-05:00	2024-03-13T12:00:00+00:00	24
2024-03-14T07:00:00-05:00	2024-03-14T12:00:00+00:00	24
2024-03-15T07:00:00-05:00	2024-03-15T12:00:00+00:00	24
2024-03-16T07:00:00-05:00	2024-03-16T12:00:00+00:00	24
2024-03-17T07:00:00-05:00	2024-03-17T12:00:00+00:00	24
2024-03-18T07:00:00-05:00	2024-03-18T12:00:00+00:00	24
2024-03-1

### Show Interval Boundaries for Current Time for All Intervals ###

In [16]:
t = HecTime.now()
t.midnight_as_2400 = False
print("Current Time\t\tIntvl\tTop of Prev\t\tTop of Next")
for intvl in Interval.get_all(lambda i: i.minutes > 0):
    prev = HecTime(t).adjust_to_interval_offset(intvl, 0)
    next = HecTime(prev).increment(1, intvl)
    print(f"{t}\t{intvl}\t{prev}\t{next}")

Current Time		Intvl	Top of Prev		Top of Next
2025-08-06T14:47:00	PT1M	2025-08-06T14:47:00	2025-08-06T14:48:00
2025-08-06T14:47:00	PT2M	2025-08-06T14:46:00	2025-08-06T14:48:00
2025-08-06T14:47:00	PT3M	2025-08-06T14:45:00	2025-08-06T14:48:00
2025-08-06T14:47:00	PT4M	2025-08-06T14:44:00	2025-08-06T14:48:00
2025-08-06T14:47:00	PT5M	2025-08-06T14:45:00	2025-08-06T14:50:00
2025-08-06T14:47:00	PT6M	2025-08-06T14:42:00	2025-08-06T14:48:00
2025-08-06T14:47:00	PT10M	2025-08-06T14:40:00	2025-08-06T14:50:00
2025-08-06T14:47:00	PT12M	2025-08-06T14:36:00	2025-08-06T14:48:00
2025-08-06T14:47:00	PT15M	2025-08-06T14:45:00	2025-08-06T15:00:00
2025-08-06T14:47:00	PT20M	2025-08-06T14:40:00	2025-08-06T15:00:00
2025-08-06T14:47:00	PT30M	2025-08-06T14:30:00	2025-08-06T15:00:00
2025-08-06T14:47:00	PT1H	2025-08-06T14:00:00	2025-08-06T15:00:00
2025-08-06T14:47:00	PT2H	2025-08-06T14:00:00	2025-08-06T16:00:00
2025-08-06T14:47:00	PT3H	2025-08-06T12:00:00	2025-08-06T15:00:00
2025-08-06T14:47:00	PT4H	2025-08-06T12:0

### Format Date With Style (Generate Date Style Table) ###

In [17]:
width = 18
t = HecTime("02Jun1985, 12:34").label_as_time_zone("local")
html = '<html><pre><table border="1"><tr><th colspan="4" style="text-align:left;"><b>Base Styles</b></th></tr>'
for row in range(10):
    html += "<tr>"
    for col in (0, 10, 100, 110):
        style = row + col
        html += f"<td><b>{style}:</b> {t.date(style).rjust(width)}</td>"
    html += "</tr>"
html += '<tr><th colspan="4" style="text-align:left;"><b>Extended Styles</b></th></tr>'
for row in (-1, -2, -3):
    html += "<tr>"
    for col in (0, 10, 100, 110):
        style = row - col
        if style in (-3, -103, -113):
            html += "<td></td>"
        else:
            html += f"<td><b>{style}:</b> {t.date(style).rjust(width)}</td>"
    html += "</tr>"
html += "</table></pre></html>"
display_html(html, raw=True)  # type: ignore

Base Styles 0: June 2, 1985 10: June 2, 85 100: JUNE 2, 1985 110: JUNE 2, 85 1: Jun 2, 1985 11: Jun 2, 85 101: JUN 2, 1985 111: JUN 2, 85 2: 2 June 1985 12: 2 June 85 102: 2 JUNE 1985 112: 2 JUNE 85 3: June 1985 13: June 85 103: JUNE 1985 113: JUNE 85 4: 02Jun1985 14: 02Jun85 104: 02JUN1985 114: 02JUN85 5: 2Jun1985 15: 2Jun85 105: 2JUN1985 115: 2JUN85 6: Jun1985 16: Jun85 106: JUN1985 116: JUN85 7: 02 Jun 1985 17: 02 Jun 85 107: 02 JUN 1985 117: 02 JUN 85 8: 2 Jun 1985 18: 2 Jun 85 108: 2 JUN 1985 118: 2 JUN 85 9: Jun 1985 19: Jun 85 109: JUN 1985 119: JUN 85 Extended Styles -1: 6/2/85 -11: 06/02/85 -101: 6/2/1985 -111: 06/02/1985 -2: 6-2-85 -12: 06-02-85 -102: 6-2-1985 -112: 06-02-1985 -13: 1985-06-02

### Format and Parse Date/Time Like `datetime.datetime` ###

In [18]:
t = HecTime("02Jun1985, 12:34").label_as_time_zone("local")
print(f"HecTime object = {t}")
format = "%A, %B %d, %Y %H:%M:%S %z"
date_time_str = t.strftime(format)
print(f'Formatted using strftime("{format}")  : {date_time_str}')
t.set_undefined()
t.label_as_time_zone(None)
print(f"\nNew HecTime Object : {repr(t)}")
print(
    f'After parsing "{date_time_str}" with "{format}"  : {t.strptime(date_time_str, format)}'
)

HecTime object = 1985-06-02T12:34:00+00:00
Formatted using strftime("%A, %B %d, %Y %H:%M:%S %z")  : Sunday, June 02, 1985 12:34:00 +0000

New HecTime Object : HecTime(UNDEFINED_TIME, MINUTE_GRANULARITY)
After parsing "Sunday, June 02, 1985 12:34:00 +0000" with "%A, %B %d, %Y %H:%M:%S %z"  : 1985-06-02T12:34:00+00:00
